In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
from tqdm import tqdm, tqdm_notebook

from konlpy.tag import Okt, Twitter, Kkma, Hannanum, Komoran

In [2]:
import nltk
from nltk.corpus import wordnet

from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TreebankWordTokenizer

from keras.preprocessing.text import text_to_word_sequence

Using TensorFlow backend.


In [204]:
# nltk.download()

In [21]:
dic_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\emotion\emotion_dic"
review_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\reply_crwaling\naver_reply"
save_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\emotion\emotion_score"
data_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\movie\movieInfo\all"

In [22]:
data = pd.read_csv(os.path.join(data_path, "movie_merged_all_en.csv"), encoding="utf-8")

In [24]:
data[data.movieCd == "20112821"]

,movieCd,movieNm,movieNmEn,prdtYear,openDt,prdtStatNm,repGenre,genre,repNation,nations,...,directors,actors,actors_en,cast,cast_en,companyCd,companyNm,companyNmEn,companyPartNm,watchGradeNm
18385,20112821,프렌즈 위드 베네핏,Friends With Benefits,2011,20111027,개봉,코미디,"코미디,멜로/로맨스",미국,미국,...,윌 글럭,"['저스틴 팀버레이크', '밀라 쿠니스']","['Justin Timberlake', 'Mila Kunis']","['딜런 ', '제이미 ']","['', '']","['20100616', '20100616']","['한국소니픽쳐스릴리징브에나비스타영화㈜', '한국소니픽쳐스릴리징브에나비스타영화㈜']","['Sony Pictures Realizing Buena Vista ', 'Sony...","['배급사', '수입사']",['청소년관람불가']


In [9]:
dic = pd.read_csv(os.path.join(dic_path, "emotion_dic_mod2.csv"), encoding="euc-kr")

In [7]:
pos_neg_dic = pd.read_csv(os.path.join(dic_path, "pos_neg_dic.csv"), encoding="euc-kr")

In [9]:
len(pos_neg_dic)

363

In [10]:
pos_neg_dic.emotion.unique()

array(['긍정', '부정'], dtype=object)

In [11]:
dic.emotion.unique()

array(['감동', '공포', '분노', '슬픔', '유쾌', '지루함'], dtype=object)

In [7]:
# titan = pd.read_csv(os.path.join(review_path, "19980074_타이타닉.csv"), encoding="utf-8")
# transformer = pd.read_csv(os.path.join(review_path, "20070061_트랜스포머.csv"), encoding="utf-8")
# avatar = pd.read_csv(os.path.join(review_path, "20090834_아바타.csv"), encoding="utf-8")
# gwanhae = pd.read_csv(os.path.join(review_path, "20124079_광해, 왕이 된 남자.csv"), encoding="utf-8")
# myungrang = pd.read_csv(os.path.join(review_path, "20129370_명량.csv"), encoding="utf-8")
# haeundae = pd.read_csv(os.path.join(review_path, "20090074_해운대.csv"), encoding="utf-8")
# interstella = pd.read_csv(os.path.join(review_path, "20149120_인터스텔라.csv"), encoding="utf-8")
# kisangchung = pd.read_csv(os.path.join(review_path, "20183782_기생충.csv"), encoding="utf-8")
# sevenbang = pd.read_csv(os.path.join(review_path, "20127593_7번방의 선물.csv"), encoding="utf-8")
# taegeuk = pd.read_csv(os.path.join(review_path, "20030371_태극기 휘날리며.csv"), encoding="utf-8")

In [8]:
# notebook = pd.read_csv(os.path.join(review_path, "20040734_노트북.csv"), encoding="utf-8")
# gouimul = pd.read_csv(os.path.join(review_path, "20060151_괴물.csv"), encoding="utf-8")
# friends = pd.read_csv(os.path.join(review_path, "20112821_프렌즈 위드 베네핏.csv"), encoding="utf-8")
# byunhoin = pd.read_csv(os.path.join(review_path, "20134803_변호인.csv"), encoding="utf-8")
# abouttime = pd.read_csv(os.path.join(review_path, "20135428_어바웃 타임.csv"), encoding="utf-8")
# winter = pd.read_csv(os.path.join(review_path, "20136803_겨울왕국.csv"), encoding="utf-8")
# busan = pd.read_csv(os.path.join(review_path, "20156564_부산행.csv"), encoding="utf-8")
# lalaland = pd.read_csv(os.path.join(review_path, "20167904_라라랜드.csv"), encoding="utf-8")
# aladin = pd.read_csv(os.path.join(review_path, "20183867_알라딘.csv"), encoding="utf-8")
# avengers = pd.read_csv(os.path.join(review_path, "20183782_기생충.csv"), encoding="utf-8")

In [4]:
cunjering =  pd.read_csv(os.path.join(review_path, "20162941_컨저링2.csv"), encoding="utf-8")

In [5]:
len(cunjering)

7880

In [35]:
def get_emotion_score(dic, data):
    
    data = data.fillna("")
    
    cnt = 0
    result = np.zeros(6)
    for review in tqdm(data.review):
        emotions = np.zeros(6)
        if len(review) > 20:
            for i in range(len(dic.word)):
                if dic.word[i] in review:
                    if dic.emotion[i] == "감동":
                        emotions[0] += 1
                    if dic.emotion[i] == "공포":
                        emotions[1] += 1
                    if dic.emotion[i] == "분노":
                        emotions[2] += 1
                    if dic.emotion[i] == "슬픔":
                        emotions[3] += 1
                    if dic.emotion[i] == "유쾌":
                        emotions[4] += 1
                    if dic.emotion[i] == "지루함":
                        emotions[5] += 1
            if emotions.sum() == 0:
                continue
            else:
                cnt += 1
                result += emotions/(emotions.sum())
    
    print(cnt)
    result = (result/result.sum()*100).astype(int)
    
    return result

In [33]:
def get_pos_neg_score(dic, data):
    
    data = data.fillna("")
    
    cnt = 0
    result = np.zeros(2)
    for review in tqdm(data.review):
        emotions = np.zeros(2)
        if len(review) > 20:
            for i in range(len(dic.word)):
                if dic.word[i] in review:
                    if dic.emotion[i] == "긍정":
                        emotions[0] += 1
                    if dic.emotion[i] == "부정":
                        emotions[1] += 1
            if emotions.sum() == 0:
                continue
            else:
                cnt += 1
                result += emotions/(emotions.sum())
    
    print(cnt)
    result = (result/result.sum()*100).astype(int)
    
    return result

In [27]:
def get_emotion_score_df(dic, review_path, movie_info):
    
    result = pd.DataFrame()
    
    for file in os.listdir(review_path):
        file_name = file.replace("'", "").replace(".csv", "").split("_")
        movieCd = int(file_name[0])
        
        try:
            movieNm = movie_info[movie_info.movieCd == int(movieCd)].movieNm.iloc[0]
        except:
            movieNm = movie_info[movie_info.movieCd == movieCd].movieNm.iloc[0]
            
        
        movie_review = pd.read_csv(os.path.join(review_path, file), encoding="utf-8")
        emotion_score = get_emotion_score(dic, movie_review)
        df = pd.DataFrame([movieCd, movieNm, *emotion_score], 
                          index=["movieCd", "movieNm", "impression", "fear", "anger", "sadness", "fun", "boredom"]).T
        result = pd.concat([result, df], axis=0)
    
    return result

In [28]:
def get_pos_neg_score_df(dic, review_path, movie_info):
    
    result = pd.DataFrame()
    
    for file in os.listdir(review_path):
        file_name = file.replace("'", "").replace(".csv", "").split("_")
        movieCd = file_name[0]
        
        try:
            movieNm = movie_info[movie_info.movieCd == int(movieCd)].movieNm.iloc[0]
        except:
            movieNm = movie_info[movie_info.movieCd == movieCd].movieNm.iloc[0]
            
        movie_review = pd.read_csv(os.path.join(review_path, file), encoding="utf-8")
        pos_neg_score = get_pos_neg_score(dic, movie_review)
        df = pd.DataFrame([movieCd, movieNm, *pos_neg_score], 
                          index=["movieCd", "movieNm", "positive", "negative"]).T
        result = pd.concat([result, df], axis=0)
    
    return result

In [ ]:
movie_score = get_emotion_score_df(dic, review_path)

In [114]:
movie_score

,movieCd,movieNm,impression,fear,anger,sadness,fun,boredom
0,19980074,타이타닉,53,1,10,21,10,3
0,20030371,태극기 휘날리며,42,2,10,30,9,4
0,20040734,노트북,66,0,4,14,11,3
0,20060151,괴물,12,10,9,19,39,8
0,20070061,트랜스포머,9,4,11,17,45,11
0,20090074,해운대,28,2,7,19,31,10
0,20090834,아바타,14,2,10,17,40,13
0,20112821,프렌즈 위드 베네핏,10,0,6,5,69,7
0,20124079,"광해, 왕이 된 남자",28,2,6,17,39,6
0,20127593,7번방의 선물,41,0,5,29,19,3


In [63]:
movie_score.to_csv(os.path.join(save_path, "movie_emtion_score.csv"), index=None, encoding="utf-8")
# movie_score.to_csv(os.path.join(save_path, "movie_emtion_score_euc.csv"), index=None, encoding="euc-kr")

In [36]:
pos_neg_score = get_pos_neg_score_df(pos_neg_dic, review_path, data)

100%|███████████████████████████████████████████████████████████████████████████| 20550/20550 [02:12<00:00, 155.30it/s]


7086


100%|███████████████████████████████████████████████████████████████████████████| 10350/10350 [01:15<00:00, 128.71it/s]


3443


100%|█████████████████████████████████████████████████████████████████████████████| 7640/7640 [01:04<00:00, 119.26it/s]


...

11651


100%|███████████████████████████████████████████████████████████████████████████| 26460/26460 [04:16<00:00, 103.10it/s]


13333


100%|███████████████████████████████████████████████████████████████████████████| 68580/68580 [10:19<00:00, 110.72it/s]


28788


In [53]:
pos_neg_score

,movieCd,movieNm,positive,negative
0,19980074,타이타닉,90,9
0,20030371,태극기 휘날리며,86,13
0,20040734,노트북,91,8
0,20060151,괴물,78,21
0,20070061,트랜스포머,79,20
0,20090074,해운대,79,20
0,20090834,아바타,77,22
0,20112821,프렌즈 위드 베네핏,87,12
0,20124079,"광해, 왕이 된 남자",86,13
0,20127593,7번방의 선물,90,9


In [60]:
pos_neg_score.positive.to_list()

[90,
 86,
 91,
 78,
 79,
 79,
 77,
 87,
 86,
 90,
 80,
 80,
 86,
 89,
 77,
 81,
 82,
 83,
 66,
 90,
 86]

In [61]:
movie_score["positive"] = pos_neg_score.positive.to_list()
movie_score["negative"] = pos_neg_score.negative.to_list()

In [62]:
movie_score

,movieCd,movieNm,impression,fear,anger,sadness,fun,boredom,positive,negative
0,19980074,타이타닉,53,1,10,21,10,3,90,9
1,20030371,태극기 휘날리며,42,2,10,30,9,4,86,13
2,20040734,노트북,66,0,4,14,11,3,91,8
3,20060151,괴물,12,10,9,19,39,8,78,21
4,20070061,트랜스포머,9,4,11,17,45,11,79,20
5,20090074,해운대,28,2,7,19,31,10,79,20
6,20090834,아바타,14,2,10,17,40,13,77,22
7,20112821,프렌즈 위드 베네핏,10,0,6,5,69,7,87,12
8,20124079,"광해, 왕이 된 남자",28,2,6,17,39,6,86,13
9,20127593,7번방의 선물,41,0,5,29,19,3,90,9


### 컨저링 2 추가

In [12]:
cunjering_score = get_emotion_score(dic, cunjering)

100%|█████████████████████████████████████████████████████████████████████████████| 7880/7880 [00:46<00:00, 169.39it/s]


5246


In [64]:
movie_score = pd.read_csv(os.path.join(save_path, "movie_emtion_score.csv"), encoding="utf-8")

In [65]:
movie_score

,movieCd,movieNm,impression,fear,anger,sadness,fun,boredom,positive,negative
0,19980074,타이타닉,53,1,10,21,10,3,90,9
1,20030371,태극기 휘날리며,42,2,10,30,9,4,86,13
2,20040734,노트북,66,0,4,14,11,3,91,8
3,20060151,괴물,12,10,9,19,39,8,78,21
4,20070061,트랜스포머,9,4,11,17,45,11,79,20
5,20090074,해운대,28,2,7,19,31,10,79,20
6,20090834,아바타,14,2,10,17,40,13,77,22
7,20112821,프렌즈 위드 베네핏,10,0,6,5,69,7,87,12
8,20124079,"광해, 왕이 된 남자",28,2,6,17,39,6,86,13
9,20127593,7번방의 선물,41,0,5,29,19,3,90,9


In [19]:
df_cunjering = pd.DataFrame(["20162941", "컨저링 2", *cunjering_score], index=["movieCd", "movieNm", "impression", "fear", "anger", "sadness", "fun", "boredom"]).T 

In [20]:
df_cunjering

,movieCd,movieNm,impression,fear,anger,sadness,fun,boredom
0,20162941,컨저링 2,1,57,5,8,22,5


In [24]:
movie_score = pd.concat([movie_score, df_cunjering], axis=0)

In [25]:
movie_score.to_csv(os.path.join(save_path, "movie_emtion_score.csv"), index=None, encoding="utf-8")

In [66]:
pd.read_csv(os.path.join(save_path, "movie_emtion_score.csv"), encoding="utf-8")

,movieCd,movieNm,impression,fear,anger,sadness,fun,boredom,positive,negative
0,19980074,타이타닉,53,1,10,21,10,3,90,9
1,20030371,태극기 휘날리며,42,2,10,30,9,4,86,13
2,20040734,노트북,66,0,4,14,11,3,91,8
3,20060151,괴물,12,10,9,19,39,8,78,21
4,20070061,트랜스포머,9,4,11,17,45,11,79,20
5,20090074,해운대,28,2,7,19,31,10,79,20
6,20090834,아바타,14,2,10,17,40,13,77,22
7,20112821,프렌즈 위드 베네핏,10,0,6,5,69,7,87,12
8,20124079,"광해, 왕이 된 남자",28,2,6,17,39,6,86,13
9,20127593,7번방의 선물,41,0,5,29,19,3,90,9


### 코드 확인

In [107]:
name = []
for file in os.listdir(review_path):
    file_name = file.replace("'", "").replace(".csv", "").split("_")
    movieCd = int(file_name[0])
    movieNm = data[data.movieCd == movieCd].movieNm.iloc[0]
    name.append(movieNm)

In [109]:
movie_score.movieNm = name

In [158]:
titan_score = get_emotion_score(dic, titan)

100%|███████████████████████████████████████████████████████████████████████████| 20550/20550 [01:09<00:00, 295.00it/s]


4934


In [159]:
df_titan = pd.DataFrame([dic.emotion.unique(), titan_score], index=["emotion", "score"]).T
df_titan

,emotion,score
0,감동,53
1,공포,1
2,분노,10
3,슬픔,21
4,유쾌,10
5,지루함,3


In [28]:
df_titan = pd.DataFrame([dic.emotion.unique(), titan_score], index=["emotion", "score"]).T
df_titan

,emotion,score
0,감동,67
1,공포,1
2,분노,9
3,슬픔,13
4,유쾌,8
5,지루함,3


In [24]:
transformer_score = get_emotion_score(dic, transformer)

100%|███████████████████████████████████████████████████████████████████████████| 22270/22270 [01:30<00:00, 245.63it/s]


5466


In [26]:
transformer_score

array([ 9,  4, 11, 17, 45, 11])

In [25]:
df_transformer = pd.DataFrame([dic.emotion.unique(), transformer_score], index=["emotion", "score"]).T
df_transformer

,emotion,score
0,감동,9
1,공포,4
2,분노,11
3,슬픔,17
4,유쾌,45
5,지루함,11


In [106]:
df_transformer = pd.DataFrame([dic.emotion.unique(), transformer_score], index=["emotion", "score"]).T
df_transformer

,emotion,score
0,감동,37
1,공포,3
2,분노,8
3,슬픔,12
4,유쾌,31
5,지루함,7


In [162]:
avatar_score = get_emotion_score(dic, avatar)

100%|███████████████████████████████████████████████████████████████████████████| 40940/40940 [02:48<00:00, 242.84it/s]


11286


In [33]:
df_avatar = pd.DataFrame([dic.emotion.unique(), avatar_score], index=["emotion", "score"]).T
df_avatar

,emotion,score
0,감동,37
1,공포,2
2,분노,9
3,슬픔,13
4,유쾌,29
5,지루함,10


In [163]:
df_avatar = pd.DataFrame([dic.emotion.unique(), avatar_score], index=["emotion", "score"]).T
df_avatar

,emotion,score
0,감동,14
1,공포,2
2,분노,10
3,슬픔,17
4,유쾌,40
5,지루함,13


In [164]:
gwanhae_score = get_emotion_score(dic, gwanhae)

100%|███████████████████████████████████████████████████████████████████████████| 27060/27060 [01:51<00:00, 242.31it/s]


10132


In [35]:
df_gwanhae = pd.DataFrame([dic.emotion.unique(), gwanhae_score], index=["emotion", "score"]).T
df_gwanhae

,emotion,score
0,감동,39
1,공포,2
2,분노,6
3,슬픔,14
4,유쾌,32
5,지루함,6


In [165]:
df_gwanhae = pd.DataFrame([dic.emotion.unique(), gwanhae_score], index=["emotion", "score"]).T
df_gwanhae

,emotion,score
0,감동,28
1,공포,2
2,분노,6
3,슬픔,17
4,유쾌,39
5,지루함,6


In [166]:
myungrang_score = get_emotion_score(dic, myungrang)

100%|███████████████████████████████████████████████████████████████████████████| 64750/64750 [06:11<00:00, 174.06it/s]


29182


In [37]:
df_myungrang = pd.DataFrame([dic.emotion.unique(), myungrang_score], index=["emotion", "score"]).T
df_myungrang

,emotion,score
0,감동,35
1,공포,5
2,분노,9
3,슬픔,16
4,유쾌,26
5,지루함,9


In [167]:
df_myungrang = pd.DataFrame([dic.emotion.unique(), myungrang_score], index=["emotion", "score"]).T
df_myungrang

,emotion,score
0,감동,23
1,공포,4
2,분노,9
3,슬픔,18
4,유쾌,33
5,지루함,10


In [168]:
haeundae_score = get_emotion_score(dic, haeundae)
interstella_score = get_emotion_score(dic, interstella)
kisangchung_score = get_emotion_score(dic, kisangchung)

100%|███████████████████████████████████████████████████████████████████████████| 25850/25850 [02:02<00:00, 210.47it/s]


10923


100%|███████████████████████████████████████████████████████████████████████████| 48520/48520 [04:25<00:00, 182.91it/s]


17267


100%|███████████████████████████████████████████████████████████████████████████| 33560/33560 [02:55<00:00, 191.57it/s]


10804


In [169]:
df_haeundae = pd.DataFrame([dic.emotion.unique(), haeundae_score], index=["emotion", "score"]).T
df_haeundae

,emotion,score
0,감동,28
1,공포,2
2,분노,7
3,슬픔,19
4,유쾌,31
5,지루함,10


In [170]:
df_interstella = pd.DataFrame([dic.emotion.unique(), interstella_score], index=["emotion", "score"]).T
df_interstella

,emotion,score
0,감동,19
1,공포,3
2,분노,13
3,슬픔,17
4,유쾌,33
5,지루함,12


In [171]:
df_kisangchung = pd.DataFrame([dic.emotion.unique(), kisangchung_score], index=["emotion", "score"]).T
df_kisangchung

,emotion,score
0,감동,3
1,공포,11
2,분노,22
3,슬픔,22
4,유쾌,31
5,지루함,8


In [172]:
sevenbang_score = get_emotion_score(dic, sevenbang)

100%|███████████████████████████████████████████████████████████████████████████| 44800/44800 [03:19<00:00, 224.98it/s]


20070


In [173]:
df_sevenbang = pd.DataFrame([dic.emotion.unique(), sevenbang_score], index=["emotion", "score"]).T
df_sevenbang

,emotion,score
0,감동,41
1,공포,0
2,분노,5
3,슬픔,29
4,유쾌,19
5,지루함,2


In [137]:
df_sevenbang = pd.DataFrame([dic.emotion.unique(), sevenbang_score], index=["emotion", "score"]).T
df_sevenbang

,emotion,score
0,감동,41
1,공포,0
2,분노,5
3,슬픔,29
4,유쾌,19
5,지루함,2


In [ ]:
# words = []
# for i in range(len(text)):
#     word = okt.morphs(text[i], stem=True)
#     words.append(word)

# def remove_end_txt(text):
#     words = []
#     for i in range(len(text)):
#         word = text[i].replace("이다", "").replace("하다", "").replace("되다", "").replace("스럽다", "").replace("나다", "").replace("거리다", "").replace("다", "")
#         words.append(word)
#     return words

In [ ]:
# words = remove_end_txt(dic.단어)

In [62]:
# for i in range(len(dic)):
#     print(kkma.pos(dic.단어[i])[0][0])

In [63]:
# for i in range(len(dic)):
#     print(kkma.pos(dic.단어[i]))

In [64]:
# kkma.tagset

In [ ]:
# okt.morphs('너무 재미가 없다', stem=True)